# OMMX Python SDK 1.5.0

[![Static Badge](https://img.shields.io/badge/GitHub_Release-Python_SDK_1.5.0-blue?logo=github)](https://github.com/Jij-Inc/ommx/releases/tag/python-1.5.0)

このノートブックでは新機能について説明します。詳細についてはGitHubのリリースノートを参照してください。

## 評価と部分評価

OMMXの最初のリリースから、`ommx.v1.Instance`は`evaluate`メソッドをサポートしており、`Solution`メッセージを生成します。

In [1]:
from ommx.v1 import Instance, DecisionVariable

# OMMX APIのインスタンスを作成
x = DecisionVariable.binary(1)
y = DecisionVariable.binary(2)

instance = Instance.from_components(
    decision_variables=[x, y],
    objective=x + y,
    constraints=[x + y <= 1],
    sense=Instance.MINIMIZE
)
solution = instance.evaluate({1: 1, 2: 0})

In [2]:
solution.decision_variables

,kind,lower,upper,name,subscripts,description,substituted_value,value
id,,,,,,,,
1,binary,0.0,1.0,<NA>,[],<NA>,<NA>,1.0
2,binary,0.0,1.0,<NA>,[],<NA>,<NA>,0.0


Python SDK 1.5.0から、`Function`とその基本クラスである`Linear`、`Quadratic`、`Polynomial`も`evaluate`メソッドをサポートします。

In [3]:
f = 2*x + 3*y
value, used_ids = f.evaluate({1: 1, 2: 0})
print(f"{value=}, {used_ids=}")

value=2.0, used_ids={1, 2}


これにより、関数の評価値と使用された意思決定変数のIDが返されます。いくつかの意思決定変数が不足している場合、`evaluate`メソッドは例外を発生させます。

In [4]:
try:
    f.evaluate({3: 1})
except RuntimeError as e:
    print(e)

Variable id (1) is not found in the solution


さらに、`partial_evaluate`メソッドもあります。

In [5]:
f2, used_ids = f.partial_evaluate({1: 1})
print(f"{f2=}, {used_ids=}")

f2=Linear(3*x2 + 2), used_ids={1}


これにより、`x = 1`を代入することで新しい関数が作成されます。`partial_evaluate`は`ommx.v1.Instance`クラスにも追加されています。

In [6]:
new_instance = instance.partial_evaluate({1: 1})
new_instance.objective

Function(x2 + 1)

このメソッドは、特定の決定変数を固定して `ommx.v1.Instance` を作成するのに役立ちます。